In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install ultralytics

In [ ]:
import cv2
import os
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from google.colab.patches import cv2_imshow
import smtplib
from email.message import EmailMessage
import mimetypes
from pathlib import Path
from datetime import datetime

# Paths
video_path = '/content/drive/Shareddrives/VIDEO/Input_Videos/ATM4.mp4'
base_output_folder = '/content/drive/Shareddrives/VIDEO/Output_Frames'
model_path = '/content/drive/Shareddrives/VIDEO/Model_Weights/Model_21_04.pt'
label_mapping = {0: 'ATM', 1: 'Human', 2: 'Tools'}

# Generate unique folder for each execution
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_folder = os.path.join(base_output_folder, f'Frames_{timestamp}')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Email Function
def send_email_with_attachment(subject, body, image_path, receiver_email, sender_email, sender_password):
    mime_type, _ = mimetypes.guess_type(image_path)
    mime_type, mime_subtype = mime_type.split('/')
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg.set_content(body)
    with open(image_path, 'rb') as img:
        msg.add_attachment(img.read(), maintype=mime_type, subtype=mime_subtype, filename=Path(image_path).name)
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)
        print("Email sent successfully!")

# Video Processing
def process_video(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    human_count = 0
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(output_folder, f"frame_{frame_index}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_index += 1

        # Load model and process frame
        model = YOLO(model_path)
        results = model(frame_path, conf=0.8)
        boxes = results[0].boxes.xyxy.tolist()
        classes = results[0].boxes.cls.tolist()

        detections = []
        for box, cls in zip(boxes, classes):
            class_name = label_mapping[int(cls)]
            if class_name == 'Tools':
                body = "Unusual activity has been detected in the surveillance . Please review the attached Snapshot for further details."
                send_email_with_attachment("Alert: Unusual Activity Detected", body, frame_path, "sahanags799@gmail.com", "unusualactivityreport@gmail.com", "bcwd yvjn lkqg cjjc")
                cap.release()
                return
            if class_name == 'Human':
                human_count += 1
                if human_count > 5:
                    body = "Unusual activity has been detected in the surveillance . Please review the attached Snapshot for further details."
                    send_email_with_attachment("Alert: Multiple Images Detected", body, frame_path, "sahanags799@gmail.com", "unusualactivityreport@gmail.com", "bcwd yvjn lkqg cjjc")
                    cap.release()
                    return

    cap.release()
    print("Video processing completed without specific detections.")

process_video(video_path, output_folder)



image 1/1 /content/drive/Shareddrives/VIDEO/Output_Frames/Frames_20240502_044043/frame_0.jpg: 480x640 (no detections), 2248.0ms
Speed: 3.3ms preprocess, 2248.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/Shareddrives/VIDEO/Output_Frames/Frames_20240502_044043/frame_1.jpg: 480x640 (no detections), 2616.7ms
Speed: 3.4ms preprocess, 2616.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/Shareddrives/VIDEO/Output_Frames/Frames_20240502_044043/frame_2.jpg: 480x640 (no detections), 2247.6ms
Speed: 2.8ms preprocess, 2247.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/Shareddrives/VIDEO/Output_Frames/Frames_20240502_044043/frame_3.jpg: 480x640 (no detections), 3156.4ms
Speed: 3.0ms preprocess, 3156.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/Shareddrives/VIDEO/Output_Frames/Frames_20240502_044043/frame_